In [1]:
#imports
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
import pandas as pd
import numpy as np
import re
from textblob import TextBlob
import itertools
import os
import tweepy as tw

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\auggi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#Read in Datafile
df = pd.read_csv('IMDBDataset.csv').to_numpy()
#temporary, cuts the dataset to 100 to save time running it when testing
df = df[:500]
size = int(df.size/2)
print(size)

500


In [3]:
#Cleans HTML Input from string
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return TextBlob(cleantext)

# This is how the Naive Bayes classifier expects the input
# removes stopwords from string
# returns a dictionary
def create_word_features(words):
    useful_words = [word for word in words if word not in stopwords.words("english")]
    my_dict = dict([(word, True) for word in useful_words])
    return my_dict

#Process Input
def findusefulwords(arr, sentiment):
    unique_words = {}
    useful_words = []

    for x in range (arr[0].size):
       unique_words[x] = set(cleanhtml(arr[0][x]).split(' '))

    for x in range (len(unique_words)):
        useful_words.append((create_word_features(unique_words[x]), sentiment))
    return useful_words

In [4]:
pos_reviews = []
neg_reviews = []

for x in range (size):
    if(df[x][1] == 'positive'):
        pos_reviews.append((df[x]))
    elif(df[x][1] == 'negative'):
        neg_reviews.append((df[x]))
pos_reviews = np.array(pos_reviews).T
pos_words = findusefulwords(pos_reviews,"positive")

neg_reviews = np.array(neg_reviews).T
neg_words = findusefulwords(neg_reviews,"negative")

In [5]:
print(len(pos_words))
print(len(neg_words))
print(pos_words[0])
print(neg_words[0])

237
263
({'prison': True, 'first': True, 'due': True, 'struck': True, 'side.': True, 'more....so': True, 'thing': True, 'forget': True, 'right': True, 'experimental': True, '1': True, 'say': True, 'taste': True, 'This': True, 'away': True, 'faint': True, 'well': True, 'lack': True, 'Watching': True, 'set': True, 'Irish': True, 'become': True, 'unflinching': True, 'City': True, 'brutality': True, 'ever': True, 'hooked.': True, 'City,': True, 'mess': True, 'home': True, 'comfortable': True, 'punches': True, 'sex': True, 'appeal': True, 'episode': True, 'agenda.': True, "who'll": True, 'section': True, 'Its': True, 'scuffles,': True, 'order': True, 'mentioned': True, 'class': True, 'inwards,': True, 'it,': True, 'mannered,': True, '(crooked': True, 'called': True, 'show': True, 'injustice': True, 'given': True, 'would': True, 'main': True, 'Trust': True, 'audiences,': True, 'happened': True, 'word.It': True, 'accustomed': True, 'Not': True, 'dare.': True, 'sold': True, 'viewing....thats':

In [6]:
train_set = neg_words[:200] + pos_words[:200]
test_set =  neg_words[200:] + pos_words[200:]
print(len(train_set),  len(test_set))

400 100


In [7]:
#Classifier
classifier = NaiveBayesClassifier.train(train_set)

In [8]:
accuracy = nltk.classify.util.accuracy(classifier, test_set)
print(accuracy * 100)

73.0


In [9]:
#Twitter API
import config
consumer_key = config.consumer_key
consumer_secret = config.consumer_secret
access_token = config.access_token
access_token_secret = config.access_token_secret
print(consumer_key)
print(consumer_secret)
print(access_token)
print(access_token_secret)
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

upZGEgcsgAdyfAjs4fL5Rb1bN
XLPRKSiLo1VmKxLGMz9OKJnEuXz2FkGs1AqT4bxQcSHpMwLZTR
1617914664-cwa4ZNpuK90CXZkMqao7qZNCnaJR6COojYttim6
dcWhdamlrzoRAKK6b2WuDjaR6DQJw64wuhTUBMc2oP7Nk


In [154]:
#Testing Twitter API
search_words = "#tezos"
date_since = "2018-11-16"

# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(5)

# Iterate and print tweets
for tweet in tweets:
    print(tweet.text)

TweepError: Twitter error response: status code = 401

In [ ]:
#https://www.pythonforengineers.com/build-a-sentiment-analysis-app-with-movie-reviews/